<a href="https://colab.research.google.com/github/shashankbhagat/Recommendation-System/blob/master/Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import tensorflow as tf
import csv
from collections import defaultdict
import random
import scipy as sc
import tensorflow as tf
#import tensorrec
!pip install tensorrec

ERROR: google-colab 1.0.0 has requirement six~=1.12.0, but you'll have six 1.11.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: six 1.12.0
    Uninstalling six-1.12.0:
      Successfully uninstalled six-1.12.0


In [8]:
print('Loading Data')
with open('ratings.csv') as ratings_file:
    ratings_file_reader=csv.reader(ratings_file)
    raw_ratings = list(ratings_file_reader)
    raw_ratings_header = raw_ratings.pop(0)

Loading Data


In [0]:
# Iterate through the input to map MovieLens IDs to new internal IDs
# The new internal IDs will be created by the defaultdict on insertion

movielens_to_internal_user_ids=defaultdict(lambda:len(movielens_to_internal_user_ids))
movielens_to_internal_item_ids=defaultdict(lambda:len(movielens_to_internal_item_ids))
for row in raw_ratings:
    row[0] = movielens_to_internal_user_ids[int(row[0])]
    row[1] = movielens_to_internal_item_ids[int(row[1])]
    row[2]=float(row[2])
n_users = len(movielens_to_internal_user_ids)
n_items=len(movielens_to_internal_item_ids)

In [0]:
random.shuffle(raw_ratings)
cutoff=int(0.8*len(raw_ratings))
train_ratings=raw_ratings[:cutoff]
test_ratings=raw_ratings[cutoff:]

In [0]:
# This method converts a list of (user, item, rating, time) to a sparse matrix
def interactions_list_to_sparse_matrix(interactions):
    user_column,item_column, ratings_column, _ = zip(*interactions)
    return sc.sparse.coo_matrix((ratings_column,(user_column,item_column)),shape=(n_users,n_items))

# Create sparse matrices of interaction data
sparse_train_ratings = interactions_list_to_sparse_matrix(train_ratings)
sparse_test_ratings = interactions_list_to_sparse_matrix(test_ratings)

In [0]:
# Construct indicator features for users and items
user_indicator_features = sc.sparse.identity(n_users)
item_indicator_features = sc.sparse.identity(n_items)

# Build a matrix factorization collaborative filter model
cf_model = tf.te

<610x610 sparse matrix of type '<class 'numpy.float64'>'
	with 610 stored elements (1 diagonals) in DIAgonal format>